In [1]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
plt.rcParams['figure.facecolor'] = 'white'
import matplotlib.colors as colors
import cmocean.cm as cmo
from glob import glob

## Extraction at single points ("moorings") is still very slow for both methods, using xarray or using CDFTOOLS/NCO

In [2]:
moorx = [540, 820, 960, 650, 1040, 840, 870, 1200, 660, 540, 1080, 1070]
moory = [1530, 1250, 1150, 1220, 1320, 1540, 1660, 1000, 1550, 1620, 1460, 1400]
moorn = ["001", "002", "003", "004", "005", "006", "007", "008", "009", "WAL", "WLS", "WSZ"]

In [3]:
CONFS = ["CREG12.L75-REF08", "CREG12.L75-FUT08"]
years = {}
years[CONFS[0]] = [y for y in range(1996, 2016)]
years[CONFS[1]] = [y for y in range(2060, 2071)]

In [ ]:
CONF = CONFS[1]
gridT_files = glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/206?/" + CONF 
                   + "_*.5d_gridT.nc") + glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/2070/" + CONF + "_*.5d_gridT.nc")
icemod_files = glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/206?/" + CONF 
                   + "_*.5d_icemod.nc") + glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/2070/" + CONF + "_*.5d_icemod.nc")
KE_files = glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-KE/206?/" + CONF 
                + "_*.5d_KE.nc") + glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-KE/2070/" + CONF + "_*.5d_KE.nc")

for i in np.arange(0, len(moorn)):
    # seems like the extraction of profiles is a bit faster when selecting the ij-point already during the preprocessing
    def preproT(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.drop(["mldkz5", "botpres", "time_centered_bounds", "time_counter_bounds", "time_centered", "deptht_bounds"])
    def preproI(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.drop(["sitemp", "sisali", "siages", "snthic", "snvolu", "iicestru", "iicestrv", "sivelo", "sivolu", "sistre", 
                        "sidive", "sishea", "intstrx", "intstry", "time_centered_bounds", "time_counter_bounds", "time_centered"])
    def preproK(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.set_coords(("nav_lat", "nav_lon"))
    moorI = f'{moorx[i]:04}'
    moorJ = f'{moory[i]:04}'
    gridT = xr.decode_cf(xr.open_mfdataset(gridT_files, chunks={"time_counter": 73, "deptht": 75}, preprocess=preproT, 
                                           decode_cf=False, coords="minimal", compat="override", parallel=True))
    icemod = xr.decode_cf(xr.open_mfdataset(icemod_files, chunks={"time_counter": 73}, preprocess=preproI, 
                                            decode_cf=False, coords="minimal", compat="override", parallel=True))
    KE = xr.decode_cf(xr.open_mfdataset(KE_files, chunks={"time_counter": 73, "deptht": 75}, preprocess=preproK, 
                                        decode_cf=False, coords="minimal", compat="override", parallel=True))
    ds = xr.merge([gridT, icemod, KE], compat="override").sel(time_counter=slice(str(years[CONF][0]) + "-01-01", str(years[CONF][-1]) + "-12-31"))
    years, datasets = zip(*ds.groupby("time_counter.year"))
    paths = [str("/data0/project/drakkar/USERS/jrieck/" + CONF + "-moorings/" + CONF + "_" + str(y) + "_moor" + moorn[i] + "_I" + moorI + "J" + moorJ 
             + "_gridT.nc") for y in years]
    xr.save_mfdataset(datasets, paths, engine="netcdf4", format="NETCDF4")
        

### bash scripts to extract points

This is slow as well. Using CDFTOOLS to extract from the 3D fields and ncks to extract from 2D fields. Not sure whether ncks could be faster for the 3D fields as well?

In [ ]:
#I=540
#J=1530
#number=001
#cd $REF08
#for year in {1996..2015}; do
#    echo $year
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        $CDF/cdfprofile -f $file -v votemper -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.votemper.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        $CDF/cdfprofile -f $file -v vosaline -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.vosaline.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        $CDF/cdfprofile -f $file -v rhop_sig0 -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.rhop_sig0.profile; done
#done
#cd $WORK/CREG12.L75-REF08-moorings/
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.votemper.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_votemper.nc
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.vosaline.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_vosaline.nc
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.rhop_sig0.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_rhop_sig0.nc
#
#I=820
#J=1250
#number=002
#cd $REF08
#for year in {1996..2015}; do
#    echo $year
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        ncks -O -d x,$I -d y,$J -v mldr10_1 $file $WORK/CREG12.L75-REF08-moorings/$file.mldr10_1.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_icemod.nc; do 
#        ncks -O -d x,$I -d y,$J -v siconc $file $WORK/CREG12.L75-REF08-moorings/$file.siconc.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_icemod.nc; do 
#        ncks -O -d x,$I -d y,$J -v sithic $file $WORK/CREG12.L75-REF08-moorings/$file.sithic.profile; done
#done
#cd $WORK/CREG12.L75-REF08-moorings/
#ncrcat -4 --cnk_dmn time_counter,1460 ????/*gridT.nc.mldr10_1.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_mldr10_1.nc
#ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.siconc.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_siconc.nc
#ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.sithic.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_sithic.nc
#
#I=540
#J=1530
#number=001
#cd $WORK/CREG12.L75-REF08-KE
#for year in {1996..2015}; do
#    echo $year
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_KE.nc; do 
#        $CDF/cdfprofile -f $file -v voke -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.voke.profile; done
#done
#
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voke.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_voke.nc

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 7))
KE(uREFmoor1, vREFmoor1).isel(depth=slice(0, 52)).mean("time_counter").plot(y="depth", yincrease=False, ax=ax[0], label="REF08")
KE(uFUTmoor1, vFUTmoor1).isel(depth=slice(0, 52)).mean("time_counter").plot(y="depth", yincrease=False, ax=ax[0], label="FUT08")

KE(uREFmoor2, vREFmoor2).isel(depth=slice(0, 52)).mean("time_counter").plot(y="depth", yincrease=False, ax=ax[1], label="REF08")
KE(uFUTmoor2, vFUTmoor2).isel(depth=slice(0, 52)).mean("time_counter").plot(y="depth", yincrease=False, ax=ax[1], label="FUT08")

KE(uREFmoor3, vREFmoor3).isel(depth=slice(0, 52)).mean("time_counter").plot(y="depth", yincrease=False, ax=ax[2], label="REF08")
KE(uFUTmoor3, vFUTmoor3).isel(depth=slice(0, 52)).mean("time_counter").plot(y="depth", yincrease=False, ax=ax[2], label="FUT08")

for a in ax:
    a.set_xlabel("KE")
    a.grid()
    a.legend(loc=4)
    
ax[0].set_xlim(-0.00002, 0.0008)
ax[1].set_xlim(-0.00002, 0.0008)
ax[2].set_xlim(-0.00002, 0.0016)
ax[0].set_ylabel("depth [m]")
ax[1].set_ylabel("")
ax[2].set_ylabel("")
ax[0].set_title("Canada Basin")
ax[1].set_title("North Pole")
ax[2].set_title("Eurasian Basin")

plt.savefig("figures/CREG12.L75_KE_REF08_2000-2009_vs_FUT08_2060-2069_moorings.png", dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 7))

n = 12
REF_colors = plt.cm.twilight_shifted(np.linspace(0, 1, n))
FUT_colors = cmo.phase(np.linspace(0, 1, n))

for m in range(0, 12):
    KE(uREFmoor1, vREFmoor1).groupby("time_counter.month").mean("time_counter").isel(depth=slice(0, 52), month=m).plot(
        y="depth", yincrease=False, ax=ax[0], label="REF08", color=REF_colors[m])
    KE(uFUTmoor1, vFUTmoor1).groupby("time_counter.month").mean("time_counter").isel(depth=slice(0, 52), month=m).plot(
        y="depth", yincrease=False, ax=ax[0], label="FUT08", color=FUT_colors[m])
    
for m in range(0, 12):
    KE(uREFmoor2, vREFmoor2).groupby("time_counter.month").mean("time_counter").isel(depth=slice(0, 52), month=m).plot(
        y="depth", yincrease=False, ax=ax[1], label="REF08", color=REF_colors[m])
    KE(uFUTmoor2, vFUTmoor2).groupby("time_counter.month").mean("time_counter").isel(depth=slice(0, 52), month=m).plot(
        y="depth", yincrease=False, ax=ax[1], label="FUT08", color=FUT_colors[m])
    
for m in range(0, 12):
    KE(uREFmoor3, vREFmoor3).groupby("time_counter.month").mean("time_counter").isel(depth=slice(0, 52), month=m).plot(
        y="depth", yincrease=False, ax=ax[2], label="REF08", color=REF_colors[m])
    KE(uFUTmoor3, vFUTmoor3).groupby("time_counter.month").mean("time_counter").isel(depth=slice(0, 52), month=m).plot(
        y="depth", yincrease=False, ax=ax[2], label="FUT08", color=FUT_colors[m])
    
for a in ax:
    a.set_xlabel("KE")
    a.grid()
    a.legend(loc=4)
    
ax[0].set_xlim(-0.00002, 0.0008)
ax[1].set_xlim(-0.00002, 0.0008)
ax[2].set_xlim(-0.00002, 0.0016)
ax[0].set_ylabel("depth [m]")
ax[1].set_ylabel("")
ax[2].set_ylabel("")
ax[0].set_title("Canada Basin")
ax[1].set_title("North Pole")
ax[2].set_title("Eurasian Basin")

#plt.savefig("figures/CREG12.L75_KE_REF08_2000-2009_vs_FUT08_2060-2069_moorings.png", dpi=300)  